In [22]:
import pandas as pd 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [23]:
val_output = pd.read_csv("val_out_df.csv")

In [24]:
val_output

,Actual,Predicted
0,0.0,0.096034
1,0.0,0.011547
2,0.0,0.031095
3,0.0,0.078262
4,0.0,0.162903
...,...,...
75493,0.0,0.729822
75494,0.0,0.376483
75495,0.0,0.017373
75496,0.0,0.002516


In [25]:
max_fp_per_scan = 8
num_thresholds = int(max_fp_per_scan * 16) + 1 # for 1/16th increments
thresholds = np.linspace(0, max_fp_per_scan, num_thresholds)



In [26]:
# Assuming 'val_output' is a DataFrame with 'Actual' and 'Predicted' columns
# Sort predictions by their score in descending order
sorted_predictions = val_output["Predicted"].sort_values(ascending=False)
sorted_labels = val_output["Actual"].loc[sorted_predictions.index]

# Calculate cumulative false positives and true positives
cumulative_false_positives = np.cumsum(1 - sorted_labels.values)
cumulative_true_positives = np.cumsum(sorted_labels.values)

# Total positives for sensitivity calculation and total scans
total_positives = sorted_labels.sum()
total_scans = len(val_output)

# Initialize arrays for FROC curve
thresholds = np.linspace(0, max_fp_per_scan, int(max_fp_per_scan * 4) + 1)
sensitivities = []
fp_per_scan = []

# Calculate sensitivity and false positives per scan for each threshold
for avg_fp_per_scan in thresholds:
    num_fp_allowed = int(avg_fp_per_scan * total_scans)
    # Get the largest index where cumulative false positives is less than or equal to num_fp_allowed
    threshold_index = np.where(cumulative_false_positives <= num_fp_allowed)[0][-1]
    
    # Sensitivity: ratio of true positives at threshold_index to total positives
    sensitivity = cumulative_true_positives[threshold_index] / total_positives
    sensitivities.append(sensitivity)
    
    # Average number of false positives per scan at the current threshold
    avg_fp = cumulative_false_positives[threshold_index] / total_scans
    fp_per_scan.append(avg_fp)

# Plot the FROC curve
plt.figure(figsize=(10, 6))
plt.plot(thresholds, sensitivities, color='#c8af76',lw=2, marker='D', markersize=6)
plt.xlabel('Average Number of False Positives per Scan', fontsize=16)
plt.ylabel('Sensitivity', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14, length=6, width=2)
plt.tick_params(axis='both', which='minor', length=4, width=1)
plt.gca().spines['top'].set_linewidth(1.5)
plt.gca().spines['right'].set_linewidth(1.5)
plt.gca().spines['bottom'].set_linewidth(1.5)
plt.gca().spines['left'].set_linewidth(1.5)
plt.grid(which="both", linestyle="-", linewidth=0.5, color="grey", alpha=0.5)
plt.savefig("images/froc.jpg", bbox_inches='tight', dpi=1200)
plt.show()


In [27]:
import pandas as pd
import numpy as np

def calculate_froc(model_output, allowed_distance, num_thresholds=40):
    # Convert the model output to a DataFrame if it's not already
    if not isinstance(model_output, pd.DataFrame):
        model_output = pd.DataFrame(model_output, columns=['Actual', 'Predicted'])

    # Initialize lists to store FROC data
    sensitivity_list = []
    fp_avg_list = []
    threshold_list = np.linspace(model_output['Predicted'].min(), model_output['Predicted'].max(), num_thresholds)

    for threshold in threshold_list:
        # Apply threshold
        predictions = model_output['Predicted'] >= threshold

        # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
        TP = ((model_output['Actual'] == 1) & predictions).sum()
        FP = ((model_output['Actual'] == 0) & predictions).sum()
        FN = ((model_output['Actual'] == 1) & ~predictions).sum()
        P = model_output['Actual'].sum() # Total number of actual positives

        # Calculate Sensitivity and False Positives per Image
        sensitivity = TP / P if P > 0 else 0
        fp_avg = FP / len(model_output) # Assuming each row is an 'image'

        # Append to lists
        sensitivity_list.append(sensitivity)
        fp_avg_list.append(fp_avg)

    return sensitivity_list, fp_avg_list, threshold_list

# Example Usage
model_output = val_output
sensitivity, fp_avg, thresholds = calculate_froc(model_output, allowed_distance=10)


In [28]:
plt.figure(figsize=(10, 6))
plt.plot(thresholds, sensitivity, color='#c8af76',lw=2, marker='D', markersize=6)
plt.xlabel('Average Number of False Positives per Scan', fontsize=16)
plt.ylabel('Sensitivity', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14, length=6, width=2)
plt.tick_params(axis='both', which='minor', length=4, width=1)
plt.gca().spines['top'].set_linewidth(1.5)
plt.gca().spines['right'].set_linewidth(1.5)
plt.gca().spines['bottom'].set_linewidth(1.5)
plt.gca().spines['left'].set_linewidth(1.5)
plt.grid(which="both", linestyle="-", linewidth=0.5, color="grey", alpha=0.5)
# plt.savefig("images/froc.jpg", bbox_inches='tight', dpi=1200)
plt.show()

In [29]:
fp_avg

[0.9979337200985456,
 0.5789159977747755,
 0.40651407984317467,
 0.3214389785159872,
 0.27039126864287794,
 0.23437706959124746,
 0.20763463932819412,
 0.18548835730747834,
 0.16747463508967125,
 0.15222919812445362,
 0.13983151871572758,
 0.12817558081008767,
 0.11828127897427747,
 0.10936713555325969,
 0.10173779437865904,
 0.09465151394738934,
 0.08837320193912422,
 0.08247900606638586,
 0.076730509417468,
 0.07153831889586479,
 0.06651831836604943,
 0.06194866089167925,
 0.05785583724072161,
 0.0533788974542372,
 0.049484754563034784,
 0.04567008397573446,
 0.042371983363797716,
 0.038424858936660575,
 0.035007549868870697,
 0.03177567617685237,
 0.028318630957111447,
 0.025470873400619883,
 0.02245092585234046,
 0.01940448753609367,
 0.01662295689952052,
 0.013920898566849454,
 0.011258576386129433,
 0.008000211926143738,
 0.004702111314206999,
 0.0]

In [30]:
import pandas as pd
import numpy as np

def calculate_froc_score(model_output, num_images, num_thresholds=1000):
    if not isinstance(model_output, pd.DataFrame):
        model_output = pd.DataFrame(model_output, columns=['Actual', 'Predicted'])

    threshold_list = np.linspace(model_output['Predicted'].min(), model_output['Predicted'].max(), num_thresholds)
    predefined_fp_rates = [1/8, 1/4, 1/2, 1, 2, 4, 8]  # Predefined false positive rates per scan
    sensitivity_at_predefined_fp_rates = []

    for fp_rate in predefined_fp_rates:
        max_sensitivity = 0
        for threshold in threshold_list:
            # Apply threshold
            predictions = model_output['Predicted'] >= threshold

            # Calculate TP, FP
            TP = ((model_output['Actual'] == 1) & predictions).sum()
            FP = ((model_output['Actual'] == 0) & predictions).sum()
            P = model_output['Actual'].sum()  # Total actual positives

            # Calculate Sensitivity and FP per Image
            sensitivity = TP / P if P > 0 else 0
            fp_per_image = FP / num_images

            # Check if FP per image is close to the predefined rate
            if fp_per_image <= fp_rate:
                max_sensitivity = max(max_sensitivity, sensitivity)

        sensitivity_at_predefined_fp_rates.append(max_sensitivity)

    # Calculate the final FROC score as the average sensitivity at predefined FP rates
    froc_score = np.mean(sensitivity_at_predefined_fp_rates)
    return froc_score, sensitivity_at_predefined_fp_rates, predefined_fp_rates

# Example Usage
model_output = val_output 
num_images =  len(val_output)
froc_score, sensitivities, fp_rates = calculate_froc_score(model_output, num_images)


In [31]:
froc_score

0.9926739926739927